<a href="https://colab.research.google.com/github/KotBaunDub/ASS-to-XLSX-converter/blob/main/SUBS_CONVERTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Профессиональный конвертер субтитров (ASS/SRT → Excel)
from google.colab import files
import re
from openpyxl import Workbook
from openpyxl.styles import Border, Side, Alignment, Font
from io import BytesIO
import os

def convert_subtitles():
    # Загрузка файла
    uploaded = files.upload()
    if not uploaded:
        print("Файл не загружен!")
        return

    # Получаем первый загруженный файл
    file_name = list(uploaded.keys())[0]
    content = uploaded[file_name].decode('utf-8')

    # Определяем формат файла
    if file_name.lower().endswith('.ass'):
        data = parse_ass(content)
    elif file_name.lower().endswith('.srt'):
        data = parse_srt(content)
    else:
        print("Ошибка: поддерживаются только .ass и .srt файлы")
        return

    # Создаем Excel с форматированием
    wb = Workbook()
    ws = wb.active
    ws.append(["Время начала", "Актер", "Текст"])

    # Стили оформления
    thin_border = Border(left=Side(style='thin'),
                        right=Side(style='thin'),
                        top=Side(style='thin'),
                        bottom=Side(style='thin'))

    center_alignment = Alignment(horizontal='center', vertical='center')
    left_alignment = Alignment(horizontal='left', vertical='center')
    bold_font = Font(bold=True)

    # Заполняем данные
    for i, (start_time, actor, text) in enumerate(data, start=2):
        ws.append([format_time(start_time), actor, text])

        # Применяем стили к ячейкам
        for col in range(1, 4):
            ws.cell(row=i, column=col).border = thin_border

        # Выравнивание
        ws.cell(row=i, column=1).alignment = center_alignment  # Время
        ws.cell(row=i, column=2).alignment = center_alignment  # Актер
        ws.cell(row=i, column=3).alignment = left_alignment    # Текст

    # Форматирование заголовков
    for col in range(1, 4):
        ws.cell(row=1, column=col).alignment = center_alignment
        ws.cell(row=1, column=col).font = bold_font
        ws.cell(row=1, column=col).border = thin_border

    # Автоподбор ширины колонок
    ws.column_dimensions['A'].width = 12  # Время
    ws.column_dimensions['B'].width = 15  # Актер
    ws.column_dimensions['C'].width = 50  # Текст

    # Сохраняем файл
    output_filename = f"converted_{os.path.splitext(file_name)[0]}.xlsx"
    wb.save(output_filename)

    # Скачиваем результат
    files.download(output_filename)

def format_time(time_str):
    """Форматирует время, удаляя миллисекунды"""
    if '.' in time_str:
        return time_str.split('.')[0]
    if ',' in time_str:
        return time_str.split(',')[0]
    return time_str

def parse_ass(content):
    """Парсинг ASS формата с правильным извлечением данных"""
    result = []
    format_line = None
    in_events = False

    for line in content.splitlines():
        line = line.strip()

        # Ищем строку с форматом
        if line.startswith("Format:"):
            format_line = line.split(":")[1].strip()
            format_fields = [f.strip() for f in format_line.split(",")]
            continue

        if line == "[Events]":
            in_events = True
            continue

        if in_events and line.startswith("Dialogue:"):
            parts = line.split(":", 1)[1].split(",", 9)  # Разделяем по запятым
            if len(parts) < 10:
                continue

            # Создаем словарь с данными по формату
            data = dict(zip(format_fields, parts))

            start_time = data.get("Start", "").strip()
            actor = data.get("Name", "").strip()  # Берем из колонки Name
            text = data.get("Text", "").strip()

            # Очищаем текст от разметки
            text = re.sub(r'\{.*?\}', '', text).replace("\\N", " ").strip()

            result.append((start_time, actor, text))

    return result

def parse_srt(content):
    """Парсинг SRT формата"""
    result = []
    blocks = re.split(r'\n\s*\n', content.strip())
    for block in blocks:
        lines = [l.strip() for l in block.split('\n') if l.strip()]
        if len(lines) >= 3:
            timecode = lines[1]
            text = ' '.join(lines[2:]).strip()
            result.append((timecode.split(' --> ')[0], "", text))  # Пустое имя актера для SRT
    return result

# Запуск конвертера
print("Загрузите файл субтитров (.ass или .srt):")
convert_subtitles()